# Topic modelling of Bundestag speeches

Dataset from https://opendiscourse.de/

> Richter, F.; Koch, P.; Franke, O.; Kraus, J.; Kuruc, F.; Thiem, A.; Högerl, J.; Heine, S.; Schöps, K. (2020). Open Discourse. https://doi.org/10.7910/DVN/FIKIBO. Harvard Dataverse. V3.

In [1]:
import numpy as np
import pandas as pd
import pyarrow.feather as feather

## Load Data

*Data in feather format https://arrow.apache.org/docs/python/feather.html*

In [2]:
speeches_df = feather.read_feather('data/opendiscourse/feather/speeches.feather')
fractions_df = feather.read_feather('data/opendiscourse/feather/factions.feather')

## Prepare Data

In [3]:
speeches_df.head()

,id,session,electoralTerm,firstName,lastName,politicianId,speechContent,factionId,documentUrl,positionShort,positionLong,date
0,0,2,1,,Köhler,11001150,Meine Damen und Herren! Ich eröffne die 2. Sit...,-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Presidium of Parliament,präsident,1949-09-12
1,1,2,1,,Arnold,-1,"Der Bundesrat ist versammelt, Herr Präsident.\n",-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Guest,präsident des bundesrats,1949-09-12
2,2,2,1,,Köhler,11001150,Ich danke für diese Erklärung. Ich stelle dami...,-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Presidium of Parliament,präsident,1949-09-12
3,3,2,1,,Heuss,-1,"Ja, ich habe den Wunsch.\n",-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Guest,bundespräsident,1949-09-12
4,4,2,1,,Köhler,11001150,Ich erteile dem Herrn Bundespräsidenten das Wo...,-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Presidium of Parliament,präsident,1949-09-12


In [4]:
fractions_df.replace(to_replace='not found', value=np.nan, inplace=True)
fractions_df.rename(columns={'abbreviation':'fraction', 'fullName':'fractionFullName'}, inplace=True)

In [5]:
fractions_df.head()

,id,fraction,fractionFullName
0,-1,NaN,NaN
1,0,AfD,Alternative für Deutschland
2,1,BHE,Block der Heimatvertriebenen und Entrechteten
3,2,BP,Bayernpartei
4,3,Grüne,Bündnis 90/Die Grünen


In [6]:
speeches_df = speeches_df.merge(fractions_df, left_on='factionId', right_on='id', how='left', suffixes=('_speech','_fraction'))

In [7]:
speeches_df.head()

,id_speech,session,electoralTerm,firstName,lastName,politicianId,speechContent,factionId,documentUrl,positionShort,positionLong,date,id_fraction,fraction,fractionFullName
0,0,2,1,,Köhler,11001150,Meine Damen und Herren! Ich eröffne die 2. Sit...,-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Presidium of Parliament,präsident,1949-09-12,-1,NaN,NaN
1,1,2,1,,Arnold,-1,"Der Bundesrat ist versammelt, Herr Präsident.\n",-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Guest,präsident des bundesrats,1949-09-12,-1,NaN,NaN
2,2,2,1,,Köhler,11001150,Ich danke für diese Erklärung. Ich stelle dami...,-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Presidium of Parliament,präsident,1949-09-12,-1,NaN,NaN
3,3,2,1,,Heuss,-1,"Ja, ich habe den Wunsch.\n",-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Guest,bundespräsident,1949-09-12,-1,NaN,NaN
4,4,2,1,,Köhler,11001150,Ich erteile dem Herrn Bundespräsidenten das Wo...,-1,https://dip21.bundestag.de/dip21/btp/01/01002.pdf,Presidium of Parliament,präsident,1949-09-12,-1,NaN,NaN
